# | WEB SCRAPING: Wuzzuf.net

## 1 - Install & Import Libraries

In [ ]:
# Libraries Installation

#!pip install lxml
#!pip install requests
#!pip install beautifulsoup4

In [3]:
# Import Libraries

import requests
from urllib.request import urlopen

from bs4 import BeautifulSoup
import csv
from itertools import zip_longest


from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options


#Optional Libraries

import datetime
import time
import smtplib


## 2- Create Data Containers

In [10]:
job_title = []
company = []
address = []
job_type = []
job_skill = []
link = []
salary = []
responsibility = []
date = []
description = []
link = []

In [12]:
# ------------------------------------------------------FETCH THE DATA----------------------------------------------------

page_num = 0


while True:
    try:
        headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}
        page = requests.get(f"https://wuzzuf.net/search/jobs/?q=data+analyst&a=hpb={page_num}", headers=headers)

        # Assign Page Source
        src = page.content
        #print(src)










        # -----------------------------------------------------PARSE THE DATA--------------------------------------------

        # Create Soup Object for Parsing

        ugly_soup = BeautifulSoup(src, "lxml")
        soup = BeautifulSoup(ugly_soup.prettify(), "lxml")
        soup_skills = BeautifulSoup(ugly_soup.prettify(), "lxml")
        #print(soup)



        #pages_available = No.Of Results / Results per Page
        No_Of_Results = int(soup.find("strong").text)


        if(page_num > No_Of_Results // 15):
            print("Pages Ended")
            break




        #------------------------Getting The Underlined Elements----------------------


        job_titles = soup.find_all("h2",{"class":"css-m604qf"})
        #print(len(job_titles))
        #print(job_titles)




        companies = soup.find_all("a",{"class":"css-17s97q8"})
        #print(len(companies))
        #print(companies)


        addresses = soup.find_all("span",{"class":"css-5wys0k"})
        #print(len(addresses))
        #print(addresses)


        job_types = soup.find_all("div",{"class":"css-1lh32fc"})
        #print(len(job_types))
        #print(job_types)



        # This line is looking for <span> elements that have either the class "css-4c4ojb" or "css-do6t5g",
        # Because the posted/date info has two different classes
        # If an element has either of these classes, it will be included in the date_elements list.

        dates = soup.find_all("div",class_=["css-4c4ojb","css-do6t5g"])
        #print(len(dates))
        #print(dates)



        #job_skills: Because job skill has un-classed tag just <div>,
        # I got in the parent tag but excluded the other child uneeded tag to inlcude only job skill un-classed tag.

        job_skills = []
        for div in soup_skills.find_all('div', {"class":'css-y4udm8'}):
            job_type_span = div.find('span', {"class":'css-1ve4b75 eoyjyou0'})
            if job_type_span:
                job_type_span.extract()
            job_skills.append(div)

        #print(len(job_skills))
        #print(job_skills)





        #------------------------Getting The Underlined Information----------------------


        # any list from the above will work not just len(jobskills) cuz they all equal to 15 indexes
        for i in range(len(job_skills)):
            job_title.append(job_titles[i].text.strip())


            #link element <a> is already a branch in job_title element (<h2> "class":"css-m604qf")\[i]: 
            link.append(job_titles[i].a.attrs["href"])




            company.append(companies[i].text.strip())

            cleaned_address = ' '.join(addresses[i].text.replace('\n', '').split())
            address.append(cleaned_address)

            cleaned_job_type = ' '.join(job_types[i].text.replace('\n', '').split())
            job_type.append(cleaned_job_type)

            cleaned_date = ' '.join(dates[i].text.replace('\n', '').split())
            date.append(cleaned_date)

            cleaned_job_skill = ' '.join(job_skills[i].text.replace('\n', '').replace('·',"|").split())
            job_skill.append(cleaned_job_skill)

        #print(job_title,company,address,job_type,date,job_skill,link)

        page_num+=1
        print("Page Switched")
    except:
        print("Error Found")
        break







#------------------------------Internal Information from  The Link Container Created-------------------------------------

# for internal description information

for link_i in link:
    #Page Fetch
    job_page = requests.get(link_i,headers=headers)
    #Page Source
    job_src = job_page.content
    #Parsing
    job_soup = BeautifulSoup(job_src, "lxml")
    
    description_element = job_soup.find("div",{"class":"css-1uobp1k"})
    
    # Doing the following to customize the Bullet Points ** in the job description
    description_text = ""
    # Here the class "class":"css-1uobp1k" had the Bullet Points in page with <p> and other pages with <li>
    for p in description_element.find_all(["p","li"]):
        description_text += "** "+p.text+"| "
    description_text = description_text[:-2]
    description.append(description_text)
        
#print(description)



# for internal Salary information

#-------- Selenium for salary cuz JavaScript-----

# Set the path to the ChromeDriver executable
driver_path = '/path/to/chromedriver'

# Set up Chrome options (optional)
chrome_options = Options()
chrome_options.add_argument('--headless')  # Run Chrome in headless mode (no GUI)

# Create a new instance of the Chrome driver with options
driver = webdriver.Chrome(options=chrome_options)

# Define the user agent (optional)

# Iterate over the first three links
for link_i in link:
    # Navigate to the URL
    driver.get(link_i)

    # Wait for Java
    #Script to load (you may need to adjust the wait time based on the website)
    driver.implicitly_wait(10)

    # Use Selenium to find elements
    salary_elements = driver.find_elements(By.XPATH, '//span[@class="css-4xky9y"]')

    # Extract information in the 4th index because the class="css-4xky9y" is not assoctiated with just SALARY.. 
    # but with Experience, Career level... , and the salary locate as the 4th index, this why I did that
    if len(salary_elements) >= 4:
        salary.append(salary_elements[3].text)
    else:
        print("Not enough elements in the list.")
#print(salary)

# Close the browser window
driver.quit()



Page Switched
Page Switched
Page Switched
Page Switched
Page Switched
Page Switched
Page Switched
Page Switched
Page Switched
Page Switched
Page Switched
Page Switched
Page Switched
Page Switched
Page Switched
Page Switched
Page Switched
Page Switched
Pages Ended


In [13]:
# -------------------------------------------------SAVE THE DATA---------------------------------------------------

## Create CSV File & Fill It

file_list = [job_title,company,description,salary,date,address,job_type,job_skill,link]
extended_file_list = zip_longest(*file_list)

with open("E:/The Journey/Career/Data Science/_Portfolio Projects/Python Projects/Web Scraping Projects/Web Scraping - bs4 Template/bs4Template.csv", "w",encoding='utf-8', newline='') as myfile:
    wr = csv.writer(myfile)
    wr.writerow(["job_title","company","description","salary","date","address","job_type","job_skill","link"])
    wr.writerows(extended_file_list)